In [1]:
from selenium.webdriver.chrome.options import Options
options = Options()
options.binary_location = "/usr/local/bin/chromedriver"
import helium as hell
from pathlib import Path
import time
import glob
import os
import sys
import getpass
#from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys

In [2]:
downloads='Downloads'
path='{}/{}'.format(os.getenv("HOME"),downloads)

In [3]:
url="https://login.webofknowledge.com"
url_login="https://login.ezproxy.unal.edu.co/login?url=http://www.webofknowledge.com"
#browser=hell.start_firefox(url_login)
browser=hell.start_chrome(url_login)

In [ ]:
user=input('user:') #or email

In [5]:
passwd=getpass.getpass('password:')

password: ·············


Configuration

In [6]:
country='Colombia'
UPDATE=False
UPDATE_from='2020'
SciELO=True

In [7]:
i=1
into_user=['Email address','usuario']
password=["Password","contraseña"]
tag_name=['Sign In','Ingresar']
extra_url=["",".ezproxy.unal.edu.co"]

In [8]:
hell.write(user,into=into_user[i])

In [9]:
hell.write(passwd,into=password[i])

<!-- 
email='restrepo@udea.edu.co'
passwd='666Opoqdh1!888'
-->

In [10]:
hell.click(tag_name[i])

In [11]:
hell.wait_until(lambda: hell.Text('Accept all').exists())

In [12]:
hell.click('Accept all')

In [38]:
#TODO → Select Scielo
if SciELO:
    br=browser.find_elements_by_class_name("select2-selection__rendered")    
    hell.click(br[0].text)
    hell.click('SciELO Citation Index')

In [13]:
hell.click('Advanced Search')

In [40]:
#time.sleep(10)
#qq=browser.find_element_by_name("currUrl")
#sid=qq.get_attribute('value').split('SID=')
#sid[-1]
#urls="https://apps.webofknowledge.com{}/WOS_AdvancedSearch_input.do?SID={}&product=WOS&search_mode=AdvancedSearch".format(extra_url[i],sid[-1])
#browser.get(urls)

In [14]:
browser.find_element_by_name("search").send_keys(f"CU={country}")

In [15]:
hell.click('Search')

In [16]:
hell.click('2020')

In [17]:
hell.click('2021')

In [18]:
hell.click('2022')

In [19]:
hell.click('Refine')

In [20]:
rr=browser.find_element_by_class_name("brand-blue")

In [21]:
results=eval(rr.text.replace(',',''))

In [22]:
results

35154

In [23]:
wostep=500
lstq=results%wostep
results=results-lstq

In [24]:
steps=list(range(1,results+wostep,wostep))

In [25]:
n=0

In [ ]:
nstep=499
slp=0.5
for step in steps[n:]:
    markFrom=step
    martkTo=step+nstep
    print(markFrom,martkTo)

    time.sleep(slp)
    hell.click('Export')
    time.sleep(slp)
    hell.click('Plain text file')

    time.sleep(slp)
    hell.click('Records from:')

    time.sleep(slp)
    hell.click('Author, Title, Source')

    time.sleep(slp)
    hell.click('Full Record and Cited References')

    if True:
        time.sleep(slp)
        kk=browser.find_element_by_name('markFrom').clear()
        kk=browser.find_element_by_name('markFrom').send_keys("{}".format(markFrom))

    if True:    
        time.sleep(slp)
        kk=browser.find_element_by_name('markTo').clear()
        kk=browser.find_element_by_name('markTo').send_keys("{}".format(martkTo))

    time.sleep(slp)
    hell.click('Export')

    hell.wait_until(lambda: not hell.Text('No more than 500 records at a time').exists(),timeout_secs=40 )

    n+=1
    #break

1 500
501 1000
1001 1500
1501 2000
2001 2500
2501 3000
3001 3500


In [58]:
browser.close()

In [89]:
n=63
n

63